In [33]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from fuzzywuzzy import fuzz


In [2]:
data = pd.read_csv('data\\steam-processed.csv')
data.head()

,Unnamed: 0,user_id,game_title,ratings
0,1,151603712,The Elder Scrolls V Skyrim,6
1,3,151603712,Fallout 4,5
2,5,151603712,Spore,4
3,7,151603712,Fallout New Vegas,4
4,9,151603712,Left 4 Dead 2,4


In [3]:
empty_games = data.copy()
unique_games = empty_games.game_title.unique()
unique_games


array(['The Elder Scrolls V Skyrim', 'Fallout 4', 'Spore', ...,
       'Space Colony', 'Life is Hard', 'Executive Assault'], dtype=object)

In [4]:
response = requests.get('http://api.steampowered.com/ISteamApps/GetAppList/v0002/')
response.status_code

200

In [5]:
list_of_games = response.json()
list_of_games.keys()

dict_keys(['applist'])

In [6]:
list_of_games['applist'].keys()

dict_keys(['apps'])

In [8]:
games_df = pd.DataFrame(list_of_games['applist']['apps'])
games_df.head()

,appid,name
0,216938,Pieterw test app76 ( 216938 )
1,660010,test2
2,660130,test3
3,1142500,Fun with Ragdolls: The Game
4,1142510,Cucumber Defense VR


In [10]:
needed_games = games_df[np.in1d(games_df.name, unique_games)]
needed_games.columns = ['appid', 'game_title']

In [17]:
needed_games.head()

,appid,game_title
3458,1156290,Home
4459,81198,Worms Crazy Golf
4765,80895,Trainz Simulator 12
4794,80820,BEEP
4797,80823,Vertex Dispenser


In [23]:
new_data = pd.merge(data, needed_games, on=['game_title'], how='left')

In [31]:
unique_games[~np.in1d(unique_games,games_df.name)].shape

(1667,)

In [30]:
new_data.loc[new_data.game_title == 'The Elder Scrolls V Skyrim', 'game_title'] = 'The Elder Scrolls V: Skyrim'
new_data.loc[new_data.game_title == 'Fallout New Vegas', 'game_title'] = 'Fallout: New Vegas'
new_data.loc[new_data.game_title == 'Dead Island Epidemic', 'game_title'] = 'Dead Island: Epidemic'


In [34]:
leftover = unique_games[~np.in1d(unique_games,games_df.name)].shape
for game in leftover:
    matches = []
    for steam_game in games_df.name:
        ratio = fuzz.ratio(game.lower(), steam_game.lower())
        if ratio >= 60:
            matches.append((steam_game, ratio))
    match_tuple = sorted(matches, key=lambda x: x[1])[::-1]
    match = match_tuple[0][0]
    print(f'Replacing {game}: to {match}')
    new_data.loc[new_data.game_title == game, 'game_title'] = match



AttributeError: 'int' object has no attribute 'lower'